In [ ]:
# Dynamic Pricing for Urban Parking Lots - Hackathon Capstone
# Compatible with pathwaycom/pathway (v0.14.3) and pathwaycom/llm-app

# Step 1: Set up the environment in Google Colab
!pip install pathway==0.14.3  # Specific version for compatibility
!pip install numpy pandas bokeh


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import pathway as pw
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import math
import time

In [ ]:
#Enable bokeh output in Colab
output_notebook()

In [ ]:
# Step 2: Load and Preprocess Data
try:
    df = pd.read_csv('dataset (1).csv')
    print("Dataset loaded successfully. Columns:", df.columns.tolist())
except FileNotFoundError:
    print("Error: 'dataset (1).csv' not found. Please upload the file to Colab.")
    raise


Dataset loaded successfully. Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


In [ ]:
# Combine date and time into a single timestamp (ISO 8601 for Pathway)
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                format='%d-%m-%Y %H:%M:%S', errors='coerce')
if df['timestamp'].isnull().any():
    print("Warning: Dropping rows with invalid timestamps.")
    df = df.dropna(subset=['timestamp'])
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')


In [ ]:
# Rename columns for consistency
df = df.rename(columns={
    'SystemCodeNumber': 'parking_lot_id',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'QueueLength': 'queue_length',
    'IsSpecialDay': 'is_special_day'
})

In [ ]:
# Encode TrafficConditionNearby numerically
traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['traffic_level'] = df['TrafficConditionNearby'].map(traffic_map)


In [ ]:
# Assign vehicle type weights
vehicle_weights = {'cycle': 0.5, 'bike': 0.75, 'car': 1.0, 'truck': 1.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_weights)

In [ ]:
# Simulate competitor prices (since not provided)
np.random.seed(42)
df['competitor_price'] = np.random.uniform(8, 12, size=len(df))

In [ ]:
# Check for missing values
print("Missing values:\n", df.isnull().sum())
if df.isnull().any().any():
    print("Warning: Filling missing values with defaults.")
    df = df.fillna({'traffic_level': 0, 'vehicle_weight': 1.0, 'competitor_price': 10.0})

Missing values:
 ID                        0
parking_lot_id            0
Capacity                  0
latitude                  0
longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
queue_length              0
is_special_day            0
LastUpdatedDate           0
LastUpdatedTime           0
timestamp                 0
traffic_level             0
vehicle_weight            0
competitor_price          0
dtype: int64


In [ ]:
# Save preprocessed data
df.to_csv('preprocessed_data.csv', index=False)
print("Preprocessed data saved to 'preprocessed_data.csv'. Sample:\n", df.head())


Preprocessed data saved to 'preprocessed_data.csv'. Sample:
    ID parking_lot_id  Capacity   latitude  longitude  Occupancy VehicleType  \
0   0    BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1    BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2    BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3    BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4    BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  queue_length  is_special_day LastUpdatedDate  \
0                    low             1               0      04-10-2016   
1                    low             1               0      04-10-2016   
2                    low             2               0      04-10-2016   
3                    low             2               0      04-10-2016   
4                    low             2               0      04-10-2016   

  LastUpdatedTime  

In [ ]:
# Step 3: Define Pathway Schema
class ParkingSchema(pw.Schema):
    timestamp: pw.DateTimeNaive
    parking_lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic_level: float
    is_special_day: int
    vehicle_weight: float
    latitude: float
    longitude: float
    competitor_price: float


In [ ]:
# Step 4: Pathway Streaming Pipeline
try:
    # Read data as a stream
    table = pw.io.csv.read(
        'preprocessed_data.csv',
        schema=ParkingSchema,
        mode='streaming'
    )
    print("Pathway table loaded successfully. Schema:", table.schema)
except Exception as e:
    print(f"Error in Pathway CSV read: {e}")
    raise

Pathway table loaded successfully. Schema: id          | timestamp       | parking_lot_id | occupancy | capacity | queue_length | traffic_level | is_special_day | vehicle_weight | latitude | longitude | competitor_price
ANY_POINTER | DATE_TIME_NAIVE | STR            | INT       | INT      | INT          | FLOAT         | INT            | FLOAT          | FLOAT    | FLOAT     | FLOAT           


In [ ]:
# Initialize base price
base_price = 10.0

In [ ]:
# Model 1: Baseline Linear Pricing
def model1_price(occupancy: int, capacity: int) -> float:
    k = 0.5
    occupancy_rate = occupancy / capacity
    price = base_price + k * occupancy_rate
    return max(5.0, min(20.0, price))

In [ ]:
# Model 2: Demand-Based Pricing
alpha, beta, gamma, delta, epsilon = 0.4, 0.3, 0.1, 0.1, 0.1
lambda_factor = 0.5
def model2_price(occupancy: int, capacity: int, queue_length: int,
                 traffic_level: float, is_special_day: int, vehicle_weight: float) -> float:
    max_queue = 15.0
    demand = (
        alpha * (occupancy / capacity) +
        beta * (queue_length / max_queue) -
        gamma * traffic_level +
        delta * is_special_day +
        epsilon * vehicle_weight
    )
    norm_demand = min(max(demand, 0.0), 1.0)
    price = base_price * (1 + lambda_factor * norm_demand)
    return max(5.0, min(20.0, price))

In [ ]:
# Model 3: Competitive Pricing
def model3_price(occupancy: int, capacity: int, queue_length: int,
                competitor_price: float, other_lots: list) -> tuple[float, str]:
    max_queue = 15.0
    demand = (occupancy / capacity + queue_length / max_queue) / 2
    competitor_factor = competitor_price / base_price
    price = base_price * (1 + 0.3 * demand + 0.2 * (competitor_factor - 1))
    price = max(5.0, min(20.0, price))
    reroute = ""
    if occupancy / capacity > 0.9 and competitor_price < base_price:
        reroute = f"Reroute to {other_lots[0]}" if other_lots else ""
    return price, reroute

In [ ]:
# Calculate distances for Model 3
lot_coords = df.groupby('parking_lot_id')[['latitude', 'longitude']].first().reset_index()
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

distances = {}
other_lots = {}
for i, lot1 in lot_coords.iterrows():
    other_lots[lot1['parking_lot_id']] = []
    for j, lot2 in lot_coords.iterrows():
        if lot1['parking_lot_id'] != lot2['parking_lot_id']:
            dist = haversine(lot1['latitude'], lot1['longitude'], lot2['latitude'], lot2['longitude'])
            distances[(lot1['parking_lot_id'], lot2['parking_lot_id'])] = dist
            if dist < 1:
                other_lots[lot1['parking_lot_id']].append(lot2['parking_lot_id'])

In [ ]:
# Apply pricing models
@pw.udf
def apply_model3_price(parking_lot_id: str, occupancy: int, capacity: int,
                      queue_length: int, competitor_price: float) -> tuple[float, str]:
    return model3_price(occupancy, capacity, queue_length, competitor_price,
                       other_lots.get(parking_lot_id, []))

try:
    table = table.with_columns(
        price_model1=pw.apply(model1_price, pw.this.occupancy, pw.this.capacity),
        price_model2=pw.apply(model2_price, pw.this.occupancy, pw.this.capacity,
                             pw.this.queue_length, pw.this.traffic_level,
                             pw.this.is_special_day, pw.this.vehicle_weight),
        price_model3=pw.apply(apply_model3_price, pw.this.parking_lot_id,
                             pw.this.occupancy, pw.this.capacity, pw.this.queue_length,
                             pw.this.competitor_price)[0],
        reroute=pw.apply(apply_model3_price, pw.this.parking_lot_id,
                         pw.this.occupancy, pw.this.capacity, pw.this.queue_length,
                         pw.this.competitor_price)[1]
    )
except Exception as e:
    print(f"Error applying pricing models: {e}")
    raise

In [ ]:
# Output results
try:
    # Debug: Inspect table content
    def on_change(row, time, is_addition):
        print(f"Pipeline output - Row: {row}, Time: {time}, Is addition: {is_addition}")
    pw.io.subscribe(table, on_change)

    # Write output to CSV
    pw.io.csv.write(table, 'output_prices.csv')
    print("Output configured to write to 'output_prices.csv'")
except Exception as e:
    print(f"Error configuring output: {e}")
    raise

Output configured to write to 'output_prices.csv'


In [ ]:
# Run the pipeline
try:
    pw.run(monitoring_level=pw.MonitoringLevel.NONE)  # Minimize logging for speed
    print("Pathway pipeline executed successfully")
    time.sleep(2)  # Ensure output is written
except Exception as e:
    print(f"Error running Pathway pipeline: {e}")
    raise

ERROR:pathway_engine.connectors:Parse error: some fields weren't found in the header (fields present in table: ["ID", "parking_lot_id", "Capacity", "latitude", "longitude", "Occupancy", "VehicleType", "TrafficConditionNearby", "queue_length", "is_special_day", "LastUpdatedDate", "LastUpdatedTime", "timestamp", "traffic_level", "vehicle_weight", "competitor_price"], fields specified in connector: ["timestamp", "parking_lot_id", "occupancy", "capacity", "queue_length", "traffic_level", "is_special_day", "vehicle_weight", "latitude", "longitude", "competitor_price"])
ERROR:pathway_engine.connectors:Parse error: some fields weren't found in the header (fields present in table: ["0", "BHMBCCMKT01", "577", "26.14453614", "91.73617216", "61", "car", "low", "1", "0", "04-10-2016", "07:59:00", "2016-10-04T07:59:00", "0", "1.0", "9.49816047538945"], fields specified in connector: ["timestamp", "parking_lot_id", "occupancy", "capacity", "queue_length", "traffic_level", "is_special_day", "vehicle_

In [2]:
# Step 5: Load Output for Visualization
# Step 4: Pathway Streaming Pipeline (output part only)
try:
    # Debug: Inspect table content
    def on_change(row, time, is_addition):
        print(f"Pipeline output - Row: {row}, Time: {time}, Is addition: {is_addition}")
    pw.io.subscribe(table, on_change)

    # Write output to CSV
    pw.io.csv.write(table, 'output_prices.csv')
    print("Output configured to write to 'output_prices.csv'")

    # Verify input data exists
    import os
    if not os.path.exists('preprocessed_data.csv'):
        raise FileNotFoundError("Input 'preprocessed_data.csv' not found")
    input_df = pd.read_csv('preprocessed_data.csv')
    if input_df.empty:
        raise ValueError("Input 'preprocessed_data.csv' is empty")
    print("Input data verified. Sample:\n", input_df.head())

    # Run the pipeline with timeout
    import time
    pw.run(monitoring_level=pw.MonitoringLevel.NONE, max_backoff=10)
    print("Pathway pipeline executed successfully")
    time.sleep(3)  # Increased delay to ensure output is written
except Exception as e:
    print(f"Error in pipeline or output writing: {e}")
    raise

Error in pipeline or output writing: name 'pw' is not defined


NameError: name 'pw' is not defined

In [ ]:
# Step 6: Visualizations with Bokeh
lot_id = 'BHMBCCMKT01'
lot_data = output_df[output_df['parking_lot_id'] == lot_id].sort_values('timestamp')

p = figure(title=f"Price Trends for {lot_id}", x_axis_type="datetime",
           x_axis_label="Time", y_axis_label="Price ($)")
p.line(lot_data['timestamp'], lot_data['price_model1'], legend_label="Model 1", color="blue")
p.line(lot_data['timestamp'], lot_data['price_model2'], legend_label="Model 2", color="green")
p.line(lot_data['timestamp'], lot_data['price_model3'], legend_label="Model 3", color="red")
p.legend.click_policy = "hide"
show(p)

p2 = figure(title=f"Occupancy and Queue for {lot_id}", x_axis_type="datetime",
            x_axis_label="Time", y_axis_label="Count")
p2.line(lot_data['timestamp'], lot_data['occupancy'], legend_label="Occupancy", color="blue")
p2.line(lot_data['timestamp'], lot_data['queue_length'], legend_label="Queue Length", color="orange")
p2.legend.click_policy = "hide"
show(p2)

# Step 7: Report
"""
# Dynamic Pricing Report

## Demand Function
- **Model 1**: Linear model: Price = 10 + 0.5 * (Occupancy/Capacity).
- **Model 2**: Demand = 0.4 * Occupancy/Capacity + 0.3 * Norm(QueueLength) - 0.1 * Norm(Traffic) + 0.1 * IsSpecialDay + 0.1 * VehicleWeight. Price = 10 * (1 + 0.5 * Norm(Demand)).
- **Model 3**: Adjusts Model 2 price with competitor prices and reroutes if occupancy > 90% and competitors are cheaper.

## Assumptions
- Competitor prices simulated ($8–$12) due to missing data.
- Vehicle weights: cycle=0.5, bike=0.75, car=1.0, truck=1.5.
- Prices bounded between $5 and $20.
- Traffic levels: low=0, average=1, high=2.

## Price Changes
- Model 1: Simple, occupancy-driven.
- Model 2: Responsive to multiple demand factors.
- Model 3: Competitive, with rerouting suggestions.

## Visualizations
- Price trends show Model 3’s competitiveness.
- Occupancy and queue plots highlight demand peaks.
"""